<a href="https://colab.research.google.com/github/amrzhd/MRISkullStripping-/blob/main/MRI_Skull_Stripping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MRI Skull Stripping

#Installing Packages

In [1]:
!pip install nibabel

In [2]:
!pip install nilearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 63.2 MB/s eta 0:00:00


In [4]:
!pip install monai

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

#Libraries Used

In [5]:
import os
import gdown
import numpy as np
import pandas as pd

# Scikit Learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# NI
import nibabel as nib
from nilearn.image import smooth_img, resample_to_img
from nilearn.masking import apply_mask

# Monai
import monai
from monai.transforms import (Compose, LoadImaged, EnsureChannelFirstd, ScaleIntensityd,
                              ResizeWithPadOrCropd, RandAxisFlipd, RandGaussianNoised, RandGibbsNoised)
from monai.data import Dataset, DataLoader
from monai.data.utils import pad_list_data_collate

# Torch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

#Extracting Data

In [6]:
# Replace this with your actual shareable link
shareable_link = 'https://drive.google.com/file/d/1AdTZhhsSkyn2vp_nn-pO0UisQZPQD1O8/view?usp=sharing'

# Extract file ID from the shareable link
file_id = shareable_link.split('/d/')[1].split('/view')[0]

# Create the direct download link
download_url = f'https://drive.google.com/uc?id={file_id}&export=download'

# Specify the output file path
output_file = 'CC_359.zip'

# Download the file
gdown.download(download_url, output_file, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1AdTZhhsSkyn2vp_nn-pO0UisQZPQD1O8&export=download
From (redirected): https://drive.google.com/uc?id=1AdTZhhsSkyn2vp_nn-pO0UisQZPQD1O8&export=download&confirm=t&uuid=cb7351ee-36df-40fa-9075-ca89df66d36b
To: /content/CC_359.zip
100%|██████████| 202M/202M [00:03<00:00, 51.1MB/s]


'CC_359.zip'

In [7]:
!unzip /content/CC_359.zip

Archive:  /content/CC_359.zip
   creating: CC_359/
   creating: CC_359/data/
  inflating: CC_359/data/CC0001_philips_15_55_M.nii.gz  
  inflating: CC_359/data/CC0001_philips_15_55_M_neg.nii.gz  
  inflating: CC_359/data/CC0002_philips_15_56_M.nii.gz  
  inflating: CC_359/data/CC0002_philips_15_56_M_neg.nii.gz  
  inflating: CC_359/data/CC0003_philips_15_63_F.nii.gz  
  inflating: CC_359/data/CC0003_philips_15_63_F_neg.nii.gz  
  inflating: CC_359/data/CC0004_philips_15_67_M.nii.gz  
  inflating: CC_359/data/CC0004_philips_15_67_M_neg.nii.gz  
  inflating: CC_359/data/CC0005_philips_15_62_M.nii.gz  
  inflating: CC_359/data/CC0005_philips_15_62_M_neg.nii.gz  
  inflating: CC_359/data/CC0006_philips_15_63_F.nii.gz  
  inflating: CC_359/data/CC0006_philips_15_63_F_neg.nii.gz  
  inflating: CC_359/data/CC0007_philips_15_62_M.nii.gz  
  inflating: CC_359/data/CC0007_philips_15_62_M_neg.nii.gz  
  inflating: CC_359/data/CC0008_philips_15_60_F.nii.gz  
  inflating: CC_359/data/CC0008_philips_

In [16]:

# Paths to the data and masks directories
data_dir = '/content/CC_359/data/'
mask_dir = '/content/CC_359/masks/'

# Mapping of group IDs to their corresponding numbers
group_numbers = {
    'CC0001': '55',
    'CC0002': '56',
    'CC0003': '63',
    'CC0004': '67',
    'CC0005': '62',
    'CC0006': '63',
    'CC0007': '62',
    'CC0008': '60',
    'CC0009': '69',
    'CC0010': '69'
}

# Lists to store image and mask file paths
data_list = []

# Function to check file existence and return paths
def check_and_add(image_path, mask_path):
    if os.path.exists(image_path) and os.path.exists(mask_path):
        data_list.append({"img": image_path, "brain_mask": mask_path})

# Loop through all groups from CC0001 to CC0010
for i in range(1, 11):
    group_id = f'CC{i:04d}'  # Group IDs (e.g., CC0001, CC0002, ..., CC0010)
    num = group_numbers[group_id]

    # Filenames for both M and F possibilities
    image_filename_M = f'{group_id}_philips_15_{num}_M.nii.gz'
    image_filename_F = f'{group_id}_philips_15_{num}_F.nii.gz'
    image_neg_filename_M = f'{group_id}_philips_15_{num}_M_neg.nii.gz'
    image_neg_filename_F = f'{group_id}_philips_15_{num}_F_neg.nii.gz'

    mask_filename_M = f'{group_id}_philips_15_{num}_M_staple.nii.gz'
    mask_filename_F = f'{group_id}_philips_15_{num}_F_staple.nii.gz'
    mask_neg_filename_M = f'{group_id}_philips_15_{num}_M_staple_neg.nii.gz'
    mask_neg_filename_F = f'{group_id}_philips_15_{num}_F_staple_neg.nii.gz'

    # Full paths
    image_path_M = os.path.join(data_dir, image_filename_M)
    image_path_F = os.path.join(data_dir, image_filename_F)
    image_neg_path_M = os.path.join(data_dir, image_neg_filename_M)
    image_neg_path_F = os.path.join(data_dir, image_neg_filename_F)

    mask_path_M = os.path.join(mask_dir, mask_filename_M)
    mask_path_F = os.path.join(mask_dir, mask_filename_F)
    mask_neg_path_M = os.path.join(mask_dir, mask_neg_filename_M)
    mask_neg_path_F = os.path.join(mask_dir, mask_neg_filename_F)

    # Check and add valid image-mask pairs
    check_and_add(image_path_M, mask_path_M)
    check_and_add(image_neg_path_M, mask_neg_path_M)
    check_and_add(image_path_F, mask_path_F)
    check_and_add(image_neg_path_F, mask_neg_path_F)

# Split data into training (80%) and testing (20%)
train_data, test_data = train_test_split(data_list, test_size=0.2, random_state=42)

# Define the transformations using MONAI
transforms = Compose([
    LoadImaged(keys=["img", "brain_mask"]),
    EnsureChannelFirstd(keys=["img", "brain_mask"]),
    ScaleIntensityd(keys=["img"], minv=0.0, maxv=1.0),
    ResizeWithPadOrCropd(keys=["img", "brain_mask"], spatial_size=(128, 128, 128)),  # Adjust the size as needed
    RandAxisFlipd(keys=["img", "brain_mask"], prob=0.2),
    RandGaussianNoised(keys=["img"], prob=0.2, mean=0.0, std=0.05),
    RandGibbsNoised(keys=["img"], prob=0.2, alpha=(0.1, 0.6))
])

# Create MONAI Datasets
train_dataset = monai.data.CacheDataset(data=train_data, transform=transforms)
test_dataset = monai.data.CacheDataset(data=test_data, transform=transforms)

# Create DataLoaders
train_dataloader = DataLoader(
    train_dataset,
    batch_size=32,  # Batch size of 32 for training
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    collate_fn=pad_list_data_collate,
    drop_last=True
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=1,  # Batch size of 1 for testing
    shuffle=False,
    num_workers=4,
    pin_memory=True,
    collate_fn=pad_list_data_collate,
    drop_last=False
)

# Function to print sample dimensions
def print_sample_dimensions(dataset, num_samples=1):
    for i in range(num_samples):
        sample = dataset[i]
        img = sample['img']
        brain_mask = sample['brain_mask']
        print(f"Sample {i+1} dimensions:")
        print(f"Image dimensions: {img.shape}")
        print(f"Mask dimensions: {brain_mask.shape}")

# Print dimensions of a few samples from train and test datasets
print("Train dataset sample dimensions:")
print_sample_dimensions(train_dataset, num_samples=1)

print("\nTest dataset sample dimensions:")
print_sample_dimensions(test_dataset, num_samples=1)

Loading dataset: 100%|██████████| 4/4 [00:03<00:00,  1.13it/s]

Train dataset sample dimensions:
Sample 1 dimensions:
Image dimensions: torch.Size([1, 128, 128, 128])
Mask dimensions: torch.Size([1, 128, 128, 128])

Test dataset sample dimensions:
Sample 1 dimensions:
Image dimensions: torch.Size([1, 128, 128, 128])
Mask dimensions: torch.Size([1, 128, 128, 128])



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


#Training Class

In [13]:
class TrainModel():
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def train_model(self, model, train_loader, learning_rate=0.001, batch_size=8, epochs=50):
        model = model.to(self.device)
        criterion = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy Loss with logits
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        # Training loop
        highest_train_dice = 0.0
        for epoch in range(epochs):
            model.train()
            running_loss = 0.0
            dice_score_total = 0.0
            total_batches = len(train_loader)

            for inputs, masks in train_loader:
                inputs = inputs.to(self.device)
                masks = masks.to(self.device)

                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, masks)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

                # Dice Score Calculation
                dice_score = self.dice_coefficient(outputs, masks)
                dice_score_total += dice_score

            epoch_loss = running_loss / total_batches
            epoch_dice = dice_score_total / total_batches

            if epoch_dice > highest_train_dice:
                highest_train_dice = epoch_dice

            print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Dice Score: {epoch_dice:.4f}")

        print("Highest Train Dice Score:", highest_train_dice)

        # Save model
        torch.save(model.state_dict(), 'unet_model.pth')
        return model

    def dice_coefficient(self, outputs, masks):
        smooth = 1e-6
        outputs = torch.sigmoid(outputs)
        outputs = (outputs > 0.5).float()
        intersection = (outputs * masks).sum(dim=(2, 3))
        union = outputs.sum(dim=(2, 3)) + masks.sum(dim=(2, 3))
        dice = (2. * intersection + smooth) / (union + smooth)
        return dice.mean().item()

#Evaluating Class

In [14]:
class EvalModel():
    def __init__(self, model):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = model.to(self.device)

    def test_model(self, test_loader):
        self.model.eval()
        dice_score_total = 0.0

        with torch.no_grad():
            for inputs, masks in test_loader:
                inputs = inputs.to(self.device)
                masks = masks.to(self.device)
                outputs = self.model(inputs)

                # Dice Score Calculation
                dice_score = self.dice_coefficient(outputs, masks)
                dice_score_total += dice_score

        average_dice_score = dice_score_total / len(test_loader)
        print("/------------------------------/")
        print(f"Test Dice Score: {average_dice_score:.4f}")
        print("/------------------------------/")
        return average_dice_score

    def dice_coefficient(self, outputs, masks):
        smooth = 1e-6
        outputs = torch.sigmoid(outputs)
        outputs = (outputs > 0.5).float()
        intersection = (outputs * masks).sum(dim=(2, 3))
        union = outputs.sum(dim=(2, 3)) + masks.sum(dim=(2, 3))
        dice = (2. * intersection + smooth) / (union + smooth)
        return dice.mean().item()


#UNet3D Model

In [ ]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv3d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv_block(x)

class DownSample(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DownSample, self).__init__()
        self.dconv = DoubleConv(in_channels, out_channels)
        self.pool = nn.MaxPool3d(kernel_size=2, stride=2)

    def forward(self, x):
        dconv = self.dconv(x)
        pool = self.pool(dconv)
        return dconv, pool

class UpSample(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UpSample, self).__init__()
        self.up = nn.ConvTranspose3d(in_channels, in_channels // 2, kernel_size=2, stride=2)
        self.dconv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        x1 = torch.cat([x1, x2], dim=1)  # Concatenate along the channel dimension
        x1 = self.dconv(x1)
        return x1

class UNet3DModel(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet3D, self).__init__()
        # Encoder (Down Sampling Path)
        self.enc1 = DownSample(in_channels, 64)
        self.enc2 = DownSample(64, 128)
        self.enc3 = DownSample(128, 256)
        self.enc4 = DownSample(256, 512)

        # Bottleneck
        self.bottleneck = DoubleConv(512, 1024)

        # Decoder (Up Sampling Path)
        self.up4 = UpSample(1024, 512)
        self.up3 = UpSample(512, 256)
        self.up2 = UpSample(256, 128)
        self.up1 = UpSample(128, 64)

        # Output Layer
        self.out_conv = nn.Conv3d(64, out_channels, kernel_size=1)

    def forward(self, x):
        # Encoder
        enc1, pool1 = self.enc1(x)
        enc2, pool2 = self.enc2(pool1)
        enc3, pool3 = self.enc3(pool2)
        enc4, pool4 = self.enc4(pool3)

        # Bottleneck
        bottleneck = self.bottleneck(pool4)

        # Decoder
        up4 = self.up4(bottleneck, enc4)
        up3 = self.up3(up4, enc3)
        up2 = self.up2(up3, enc2)
        up1 = self.up1(up2, enc1)

        # Output Layer
        out = self.out_conv(up1)
        return out


##Model Summery

In [ ]:
input_size = (1, 128, 128, 128)
unet3d_model = UNet3DModel().to(device)
summary(unet3d_model, input_size)

##Training Model

In [ ]:
unet3d_model = UNet3DModel().to(device)

# Training Hyperparameters
EPOCHS = 500
LEARNING_RATE = 0.001

unet3d_model = train_model(unet3d_model, train_loader,
                                           learning_rate=LEARNING_RATE, epochs=EPOCHS)
torch.save(trained_unet3d_model.state_dict(), 'unet3d_model.pth')


##Evaluating Model

In [ ]:
eval_model = EvalModel(trained_unet3d_model)
test_accuracy = eval_model.test_model(test_loader)